In [19]:
%%writefile mapper1.py

import sys
import re

def read_stopwords(file_path):
    return set(word.strip() for word in open(file_path))

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stop_words = read_stopwords('stop_words_en.txt')
word_dictionary = {}
i = 0;
article = None
word_count = 0
current_word = None
total_words = 0
for line in sys.stdin:
    if i > 0:
        break
    try:
        article, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    total_words = 0

    new_words = []
    for word in words:
        word = word.strip().lower()
        if word:
            if word in stop_words:
                continue
            new_words.append(word)
   
    total_words = len(new_words)
    new_words.sort()
    word_count = 0;
    current_word = None

    for word in new_words:
        if not word.isalpha():
            continue
        if current_word != word:
            if current_word:
                tf = float(word_count)/total_words
                print ("%s\t%s\t%f" % (current_word, article, tf))
            current_word = word
            word_count = 0;
        word_count += 1
        
if current_word:
    tf = float(word_count/total_words)
    print ("%s\t%s\t%f" % (current_word, article, tf))


Overwriting mapper1.py


In [20]:
%%writefile reducer1.py

import sys
import math

current_article = None
current_word = None
article_count = 0
word_dictionary = {}
for line in sys.stdin:
    try:
        word, article, tf = line.strip().split('\t', 2)
        tf = float(tf)
    except ValueError as e:
        continue
        
    if current_word != word:
        if current_word:
            idf = float(1)/math.log(1 + article_count)
            for a in word_dictionary:
                tff = float(word_dictionary[a])
                print ("%s\t%s\t%f\t%f\t%f" % (current_word, a, tff, idf, tff*idf))
        word_dictionary.clear()
        article_count = 0
        current_word = word
        
    article_count += 1;
    word_dictionary[article] = tf
    
if current_word:
    idf = float(1)/math.log(1 + article_count)
    for a in word_dictionary:
        tf = float(word_dictionary[a])
        print ("%s\t%s\t%f\t%f\t%f" % (current_word, a, tf, idf, tf*idf))


Overwriting reducer1.py


In [18]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 

hdfs dfs -cat ${OUT_DIR}/part* | grep -w "labor" | grep -w "12" | cut -f 5


0.000351


18/07/20 16:01:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/07/20 16:01:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/07/20 16:01:30 INFO mapred.FileInputFormat: Total input files to process : 1
18/07/20 16:01:30 INFO mapreduce.JobSubmitter: number of splits:2
18/07/20 16:01:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1532067686572_0020
18/07/20 16:01:30 INFO impl.YarnClientImpl: Submitted application application_1532067686572_0020
18/07/20 16:01:31 INFO mapreduce.Job: The url to track the job: http://38df42ab9577:8088/proxy/application_1532067686572_0020/
18/07/20 16:01:31 INFO mapreduce.Job: Running job: job_1532067686572_0020
18/07/20 16:01:40 INFO mapreduce.Job: Job job_1532067686572_0020 running in uber mode : false
18/07/20 16:01:40 INFO mapreduce.Job:  map 0% reduce 0%
18/07/20 16:02:05 INFO mapreduce.Job: Task Id : attempt_1532067686572_0020_m_000000_0, Status : FAILED
Container [pid=21010,containerID=c